<a href="https://colab.research.google.com/github/AlexeyTri/PROJECT_SCOOL/blob/master/ActualGraphic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Условия:**
1. наличие AUTODESK DESKTOP
2. доступ ко всем таблицам в режиме РЕДАКТОР
3. все тактические графики должны быть приведены в гугл форму

# TODO

* парсинг таблицы ГРАФИК ЗАКАЗЧИКА из АСС
* парсинг договора подрядчика и автоматическая генерация ГУГЛ таблиц
* каждому виду работ в контрактном графике необходимо присвоить НАШ шифр
* график заказчика подгружается с локального компьютера и сохраняется туда же
* приветсти все данные к НОРМАЛЬНОМУ ТИПУ, включая индексы и названия колонок
* разобраться с процентами

# Библиотеки

In [ ]:
!pip install pygsheets

In [2]:
import pygsheets

In [ ]:
import openpyxl
from openpyxl.cell.cell import WriteOnlyCell
from openpyxl.utils.dataframe import dataframe_to_rows
from openpyxl import Workbook
import pandas as pd
import gspread_dataframe as gd

In [ ]:
idx = pd.IndexSlice

In [ ]:
from google.colab import auth
auth.authenticate_user()
import re
import gspread
from google.auth import default
creds, _ = default()
import numpy as np
gc = gspread.authorize(creds)
from datetime import datetime

In [ ]:
start_project = '01.02.2022'
finish_project = '02.09.2024'

# **Обновление графика Закачика**

* позиции имеющие шифр обновляются по шифру
* позиции без шифра обновляются после фильтрации по шифру

## предобработка графика Заказчика

In [ ]:
# файл загружается на ГУГЛ диск
from google.colab import files
upload = files.upload()

In [ ]:
# содежание текущей папки на ГУГЛ диске
!ls

ПАНДАС из редакции графика **заказчика**

In [ ]:
df_graph = pd.read_excel(open('Сводный план-график 2022 12 28 иса.xlsx', 'rb'))

/usr/local/lib/python3.8/dist-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Conditional Formatting extension is not supported and will be removed
  warn(msg)


In [ ]:
df_graph

In [ ]:
# stop
df_graph = pd.DataFrame(df_graph.values, columns=df_graph.values[3])
df_graph['Название задачи'] = df_graph['Название задачи'].map(lambda z: str(z).lstrip()).values
df_graph

In [ ]:
# stop
df_graph.drop([0,1,2,3], inplace=True)

In [ ]:
#  список из графика заказчика
list2 = df_graph['Unnamed: 13']
list2 = list2.map(lambda z: str(z).lstrip())
list2 = list2.to_list()
list3 = df_graph['Unnamed: 2']
list3 = list3.map(lambda z: str(z).lstrip())
list3 = list3.to_list()

## обновление СМР

* пайплайн по каждому контракту СМР одинаковый

### предобработка графика работ подрядчика

In [ ]:
worksheet_SMR = gc.open('План работ Договор Адамант')

In [ ]:
graf = worksheet_SMR.worksheet("ГРАФИК РАБОТ")
graf = graf.get_all_values()
graf = np.array(graf)
graf =  pd.DataFrame(graf[1:, :])
graf

,0,1,2,3,4,5,6,7,8,9,...,282,283,284,285,286,287,288,289,290,291
0,PLAN,ГРАФИК РАБОТ,Начало работ по ГРП,Завершение работ по ГРП,Длительность,Ед.изм.,Объем по контракту,План на 17.01,Факт на 17.01,% выполнения плана,...,23.02.2023,23.02.2023,24.02.2023,24.02.2023,25.02.2023,25.02.2023,26.02.2023,26.02.2023,27.02.2023,27.02.2023
1,***,,,,,,,,,,...,план,факт,план,факт,план,факт,план,факт,план,факт
2,VZO_POS1_POD_MOB_.,Мобилизация,10.10.2022,10.05.2024,579,--,,,,"91,67%",...,,,,,,,,,,
3,VZO_POS1_POD_VRD_DP17.01,Временные дороги из дорожных плит (3х1.75),26.10.2022,18.02.2023,116,м2,300,"300,0","300,0","100,00%",...,,,,,,,,,,
4,VZO_POS1_POD_VRS_ES0.01,Временные электросети - установленные на начал...,23.11.2022,12.12.2022,20,комплект,2,"2,0","1,0","50,00%",...,,,,,,,,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
396,BOT_GP___.,Работы по благоустройству,18.11.2022,05.04.2024,505,,,"0,0","0,0",,...,,,,,,,,,,
397,BOT_GP,Выдача рабочего проекта по разделу,18.11.2022,18.11.2022,1,РД,,,,,...,,,,,,,,,,
398,_EXP___.,Ввод объекта в эксплуатацию,14.03.2024,06.06.2024,85,,,"0,0","0,0",,...,,,,,,,,,,
399,_ZOS___.,ЗОС,14.03.2024,24.04.2024,42,,,"0,0","0,0",,...,,,,,,,,,,


In [ ]:
graf_black = graf.copy()

### пересечение списков видов работ

In [ ]:
#  список из плана работ
list1 = graf.iloc[:, 0].values
list1

Пересечение списков задач плана работ и графика заказчика

In [ ]:
result = list(set(list1) & set(list2))
result

Обновляем данные в плане заказчика

In [ ]:
for i in result:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = (pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100)
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: pd.to_numeric(graf.loc[graf.iloc[:,0].map(lambda z: str(z).lstrip()).values == i, 9].values[0].replace(',', '.').replace('%', ''))/100})    

## обновление проектирования

* первый этап - обновить по шифрам
* второй этап - обновить по наименованию работ

In [ ]:
worksheet_PD = gc.open('Реестр выдачи РД')

In [ ]:
graph_PD = worksheet_PD.worksheet('График ПИР ')
graph_PD = graph_PD.get_all_values()
graph_PD = np.array(graph_PD)
graph_PD = pd.DataFrame(graph_PD[7:])
graph_PD

In [ ]:
pd.Series(graph_PD.iloc[:,7]).unique()

In [ ]:
perevod = {
    'STATUS A': 1,
    'STATUS B': 0.85,
    'STATUS C': 0.6,
    'WORK': 0.3,
    'START': 0.05,
    'HOLD': 0,
    '': ''
}

In [ ]:
list_PD_ID_1 = graph_PD[graph_PD.iloc[:, 1].values != ''].iloc[:,2]
list_PD_name = graph_PD[graph_PD.iloc[:, 1].values == ''].iloc[:,3]
list_PD = list_PD_ID_1.append(list_PD_name)
list_PD

In [ ]:
result_PD_ID = list(set(list_PD) & set(list2))
result_PD_name = list(set(list_PD) & set(list3))

In [ ]:
for i in result_PD_ID:
    df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]
    # проверка корректности внесения обновлений
    print({df_graph.loc[df_graph['Unnamed: 13'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,2] == i,7].values[0]]})

In [ ]:
for i in result_PD_name:
    df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 4'] = perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]
    print({df_graph.loc[df_graph['Unnamed: 2'].map(lambda z: str(z).lstrip()).values == i, 'Unnamed: 2'].values[0]: perevod[graph_PD.loc[graph_PD.iloc[:,3] == i,7].values[0]]}) 

In [ ]:
df_graph.to_excel('Сводный план-график {} иса.xlsx'.format('11.01.2023'), sheet_name='Обновление от {}'.format('11.01.2023'),index=False)

# **Обновление графика проектирования подрядчика**

* выгрузка реестра замечаний 
* агрегирование данных по разделам
* заполнение графика проектирования

In [ ]:
# файл с реестром замечаний на google drive
from google.colab import files
upload = files.upload()

In [ ]:
value_essuse = pd.read_excel(open('Сводка по проблемам-202301121116.xlsx', 'rb'), sheet_name=1)
value_essuse.columns

In [ ]:
value_essuse = value_essuse[(value_essuse['Статус'] == 'Открыто') & (value_essuse['Тип'].isin(['Замечание к РД','Замечания к ПД']))]
value_essuse.loc[:, 'Размещение'] =  value_essuse['Размещение'].map(lambda x: str(x).split('.pdf')[0].split('_')[0])

In [ ]:
value_essuse = value_essuse.loc[:, idx['Размещение','Тип замечания']]

In [ ]:
agg_func_count = {'Тип замечания': 'count'}
value_essuse = value_essuse.groupby(['Размещение','Тип замечания']).agg(agg_func_count)

In [ ]:
value_essuse

In [ ]:
name_RD_issues = pd.Series(value_essuse.index.get_level_values(0)).unique()

In [ ]:
graph_PD_RD = pd.DataFrame.from_records(worksheet_PD.worksheet('График ПИР ').get_all_values())

In [ ]:
list_PD_RD = graph_PD_RD.iloc[:,2].to_list()

In [ ]:
for i in name_RD_issues:
    try:
        number_row = list_PD_RD.index(i) + 1
        try:
            value_essuse.loc[i, 'Критическое замечание']
            worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS C')
        except: 
            try:
                value_essuse.loc[i, 'Техническая ошибка'].values[0] != 0  
                value_essuse.loc[i, 'Недостаточно данных'].values[0] != 0
                worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS B')
            except:
                try:
                    worksheet_PD.worksheet('График ПИР ').update_cell(number_row, 8, 'STATUS A') 
                except:
                    pass
    except:
        pass 

# **Обновление графика СМР подрядчика**

* в модели отмечается выполненный элемент
* данные попадают в GS + обновляется формы PLAN RADAR

## срез на отчетную неделю, план на следующую неделю за отчетной, кумулятивная сумма

In [ ]:
# предобработка графика работ
new_graf = graf.copy()
new_graf = new_graf.loc[idx[2:],idx[:6]]
new_graf.drop([4,5], axis=1, inplace=True)

In [ ]:
new_graf.rename(columns = {0:'passwordCOST', 
                       1:'NAME',2:'START', 3:'FINISH', 6:'valueModel'}, 
            inplace = True)
new_graf = new_graf[new_graf['passwordCOST'] > '']
new_graf = new_graf[new_graf['START'] > '']
new_graf = new_graf[new_graf['FINISH'] > '']
new_graf = new_graf.set_index('passwordCOST')
new_graf['valueModel'] = new_graf['valueModel'].map(lambda x: pd.to_numeric(re.sub("[^0-9+,]", "", x).replace(',','.')))
new_graf = new_graf.fillna(0)

In [ ]:
new_graf

In [ ]:
start_week = '30.12.2022'
start_week = pd.to_datetime(start_week)
finish_week = '16.01.2023'
finish_week = pd.to_datetime(finish_week)
# ----------------------------------------------------
start_next_week = '17.01.2023'
start_next_week = pd.to_datetime(start_next_week)
finish_next_week = '20.01.2023'
finish__next_week = pd.to_datetime(finish_next_week)
# -----------------------------------------------------
start_project = '01.02.2022'
start_project = pd.to_datetime(start_project)
finish_week = '16.01.2023'
finish_week = pd.to_datetime(finish_week)

In [ ]:
graf_week_plan = graf.loc[:, idx[start_week:finish_week,'план']].sum(axis=1).values
graf_week_plan = pd.DataFrame(graf_week_plan,index=graf.index, columns=['PLAN_WEEK'])
graf_week_fakt = graf.loc[:, idx[start_week:finish_week,'факт']].sum(axis=1).values
graf_week_fakt = pd.DataFrame(graf_week_fakt,index=graf.index, columns=['FAKT_WEEK'])

In [ ]:
graf_next_week_plan = graf.loc[:, idx[start_next_week:finish_next_week,'план']].sum(axis=1).values
graf_next_week_plan = pd.DataFrame(graf_next_week_plan,index=graf.index, columns=['PLAN_NEXT_WEEK'])
graf_next_week_fakt = graf.loc[:, idx[start_next_week:finish_next_week,'факт']].sum(axis=1).values
graf_next_week_fakt = pd.DataFrame(graf_next_week_fakt,index=graf.index, columns=['FAKT_NEXT_WEEK'])

In [ ]:
total_plan = graf.loc[:, idx[start_project:finish_week,'план']].sum(axis=1).values
total_plan = pd.DataFrame(total_plan,index=graf.index, columns=['TOTAL_PLAN'])
total_fakt = graf.loc[:, idx[start_project:finish_week,'факт']].sum(axis=1).values
total_fakt = pd.DataFrame(total_fakt,index=graf.index, columns=['TOTAL_FAKT'])

In [ ]:
new_graf_week = new_graf.join([total_plan,total_fakt,graf_week_plan,graf_week_fakt,graf_next_week_plan,graf_next_week_fakt])

In [ ]:
new_graf_week = new_graf_week[['NAME', 'TOTAL_PLAN','TOTAL_FAKT','PLAN_WEEK','FAKT_WEEK','PLAN_NEXT_WEEK','FAKT_NEXT_WEEK']]

In [ ]:
new_graf_week = new_graf_week.loc[(new_graf_week['PLAN_WEEK']>0) | (new_graf_week['PLAN_NEXT_WEEK']>0)]

In [ ]:
new_graf_week['%TOTAl_WEEK'] =( new_graf_week['FAKT_WEEK'] / new_graf_week['PLAN_WEEK']).mul(100).round(2).astype(str).add(' %')

In [ ]:
new_graf_week['%TOTAL'] =( new_graf_week['TOTAL_FAKT'] / new_graf_week['TOTAL_PLAN']).mul(100).round(2).astype(str).add(' %')

In [ ]:
new_graf_week.rename(columns={'NAME':f'WORK on {pd.to_datetime(start_week).date()}  to {pd.to_datetime(finish_week).date()}'}, inplace=True)

In [ ]:
new_graf_week

In [ ]:
worksheet_SMR.worksheet('PLAN_WEEK').clear()
worksheet_SMR.worksheet('PLAN_WEEK').append_rows([new_graf_week.columns.values.tolist()]+new_graf_week.values.tolist())

In [ ]:
# справочно
dir(worksheet_SMR.worksheet('PLAN_WEEK'))

## данные по старту работ

In [ ]:
graf_name = graf[[0,1]][2:]
graf_name.rename(columns={0: 'passwordCOST', 1: 'NAME'}, inplace=True) 
graf_name = graf_name[graf_name['passwordCOST'] != 0]
graf_name = pd.DataFrame(graf_name['NAME'].values, index=graf_name['passwordCOST'], columns=['NAME'])

In [ ]:
graf.drop(graf.index[1:10], axis=1, inplace=True)

In [ ]:
newColumnsIndex = [pd.to_datetime(pd.Series(graf.iloc[0].values[1:]).apply(lambda x: str(x)), format='%d.%m.%Y'), graf.iloc[1][1:].values]

In [ ]:
graf.rename(columns={0: 'passwordCOST'}, inplace=True)

In [ ]:
graf

In [ ]:
graf = graf.set_index("passwordCOST")
graf = pd.DataFrame(graf.values, index=graf.index, columns=newColumnsIndex)
graf.drop(graf.index[:2], axis=0, inplace=True)
for i in graf.columns:
    graf[i] = graf[i].map(lambda x: pd.to_numeric(re.sub("[^0-9 +,]", "", x).replace(',','.')))

In [ ]:
graf.fillna(0, inplace=True)
graf.sort_index(axis=0, level=1, ascending=True)
graf = graf[graf.index > '']
graf

In [ ]:
start = start_project
start = pd.to_datetime(start)
finish = pd.to_datetime(pd.Timestamp.today().date())
start, finish

In [ ]:
finish_yesterday = finish - pd.Timedelta(1, "d")

In [ ]:
total_plan = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values
cum_sum_yesterday = graf.loc[:, idx[start:finish_yesterday,'план']].sum(axis=1).values
sum_today = graf.loc[:, idx[finish:finish,'план']].sum(axis=1).values
# total_plan_value = graf.loc[:, idx[start:finish,'план']].sum(axis=1).values - graf.loc[:, idx[start:finish,'план']].sum(axis=1).values

In [ ]:
total_plan = pd.DataFrame(total_plan,index=graf.index, columns=['cum_sum_today'])
cum_sum_yesterday = pd.DataFrame(cum_sum_yesterday,index=graf.index, columns=['cum_sum_yesterday'])
sum_today = pd.DataFrame(sum_today,index=graf.index, columns=[f'sum_today {pd.to_datetime(finish).date()}'])
total_plan = total_plan.join([cum_sum_yesterday, sum_today])

In [ ]:
total_plan = total_plan[(total_plan[f'sum_today {pd.to_datetime(finish).date()}']>0) & (total_plan['cum_sum_yesterday'] == 0)]

In [ ]:
total_plan = total_plan[[f'sum_today {pd.to_datetime(finish).date()}']]

In [ ]:
worksheet_SMR.worksheet('START').clear()
gd.set_with_dataframe(worksheet_SMR.worksheet('START'), total_plan,row=1, col=1, include_index=True, include_column_header=True)

## обновление листа ПЛАН-РАБОТ-> внесение факта

In [ ]:
plan_PR = worksheet_SMR.worksheet('PR (копия)')
plan_PR = plan_PR.get_all_values()
plan_PR = np.array(plan_PR)
plan_PR = pd.DataFrame(plan_PR)
plan_PR = plan_PR.iloc[1:, [0,7,9]]
plan_PR = pd.DataFrame(plan_PR)

In [ ]:
plan_PR

,0,7,9
1,VZO_POS1_POD_VRD_DP17.04,88.0,2023-01-19
2,VZO_POS1_POD_VRD_DP17.02,120.0,2023-01-20
3,VZO_POS1_POD_VRD_DP17.01,160.0,2023-01-21
4,OOS_KZ01_BK0_BFP_OAB30.02,577.59,2023-01-22


In [ ]:
list_date = list(graf_black.iloc[0, :].values)
list_name = list(graf_black.iloc[:, 0].values)

In [ ]:
for i in plan_PR.iloc[:, 0]:
    index_in_plan_PR = list(plan_PR.iloc[:, 0]).index(i)
    date_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 2]
    date_in_plan_PR = '.'.join(reversed(str(date_in_plan_PR).split('-')))
    value_in_plan_PR = plan_PR.iloc[index_in_plan_PR, 1]
    value_in_plan_PR = ','.join(value_in_plan_PR.split('.'))

    index_row_in_graf_black = list_name.index(i) + 2
    index_column_in_graf_black = list_date.index(date_in_plan_PR) + 1

    worksheet_SMR.worksheet("ГРАФИК РАБОТ").update_cell(index_row_in_graf_black, index_column_in_graf_black, value_in_plan_PR)

    print(value_in_plan_PR, date_in_plan_PR, i, index_row_in_graf_black, index_column_in_graf_black)

## обновление листа финансирование:

1. внесение плана КС, кумулятивно за отчетный период
2. акутулизация плана платежей
3. внесение сумм по фактически выполненным работам

## графики:

1. СМР план/факт
2. финансирование план/факт
3. освоение план/факт


In [4]:
from pygsheets.chart import ChartType

In [5]:
gz = pygsheets.authorize(service_file='/content/credentials.json')

In [13]:
 gz = gz.open('План работ Договор Адамант')

In [15]:
gz.worksheet_by_title('PLAN_WEEK').add_chart(('A2', 'A44'), [('B2', 'B44')], chart_type=ChartType.LINE, title='MyChart', anchor_cell='A50')

<Chart LINE 'MyChart'>